In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

# Data Preparation

## "Best Value"

## 0 - 30 hotels

In [12]:
html_main = requests.get("https://www.tripadvisor.com/Hotels-g32655-Los_Angeles_California-Hotels.html")
bsobj_main = soup(html_main.content, "lxml")

In [13]:
rank = []
hotel_name = []
rate_score = []
hotel_link = []
for s1 in bsobj_main.find_all("div", class_ = "ui_column is-8 main_col allowEllipsis"):
    if s1.find("span", class_ = "ui_merchandising_pill sponsored_v2") == None:
        for s2 in s1.find_all("div", class_ = "listing_title"):
            hotel_name.append(s1.a.text[6:])
            hotel_link.append(s1.a["href"])
        for s3 in s1.find_all("div", class_ = "info-col"):
            if s3.a.text == "0 reviews":
                rate_score.append("0 of 5 bubbles")
            else:
                rate_score.append(s3.a["alt"])

for s4 in bsobj_main.find_all("div", class_ = "popindex"):    
    rank.append(s4.text.split(" ")[0][1:])

## The rest of the hotels - 450 recommended hotels in Los Angeles

In [14]:
top_600 = list(range(30, 600, 30))

In [15]:
for num in top_600:
    html_main = requests.get("https://www.tripadvisor.com/Hotels-g32655-oa" + str(num) + "-Los_Angeles_California-Hotels.html")
    bsobj_main = soup(html_main.content, "lxml")
    
    for s1 in bsobj_main.find_all("div", class_ = "ui_column is-8 main_col allowEllipsis"):
        if s1.find("span", class_ = "ui_merchandising_pill sponsored_v2") == None:
            for s2 in s1.find_all("div", class_ = "listing_title"):
                hotel_name.append(s1.a.text[6:])
                hotel_link.append(s1.a["href"])
            for s3 in s1.find_all("div", class_ = "info-col"):
                if s3.a.text == "0 reviews":
                    rate_score.append("0 of 5 bubbles")
                else:
                    rate_score.append(s3.a["alt"])

    for s4 in bsobj_main.find_all("div", class_ = "popindex"):    
        rank.append(s4.text.split(" ")[0][1:])

### 450 recommended hotels in Los Angeles!

In [27]:
hotel_link_update = []
for site in hotel_link:
    hotel_link_update.append("https://www.tripadvisor.com/" + site)

In [28]:
data = {"Rank": rank[:450], "Hotel Name": hotel_name[:450], "Rate": rate_score[:450], "Site Link": hotel_link_update[:450]}
df = pd.DataFrame.from_dict(data)
df

,Rank,Hotel Name,Rate,Site Link
0,1,JW Marriott Los Angeles L.A. LIVE,4.5 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
1,2,Sheraton Gateway Los Angeles Hotel,4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
2,3,Renaissance Los Angeles Airport Hotel,4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
3,4,"The Westin Bonaventure Hotel & Suites, Los Ang...",4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
4,5,"SLS Hotel, a Luxury Collection Hotel, Beverly ...",4 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
...,...,...,...,...
445,446,Downtown LA Stylish,0 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
446,447,Silver Box Beverly Hills Apartment,0 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
447,448,LA R USA Hostel,0 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...
448,449,Home Base Hostel,0 of 5 bubbles,https://www.tripadvisor.com//Hotel_Review-g326...


In [29]:
df.to_csv("450_hotel.csv")

# Location After Input

In [32]:
hotel_name[:450]

['JW Marriott Los Angeles L.A. LIVE',
 'Sheraton Gateway Los Angeles Hotel',
 'Renaissance Los Angeles Airport Hotel',
 'The Westin Bonaventure Hotel & Suites, Los Angeles',
 'SLS Hotel, a Luxury Collection Hotel, Beverly Hills',
 'The Westin Los Angeles Airport',
 'W Hollywood',
 'Residence Inn by Marriott Los Angeles LAX',
 'The Ritz-Carlton, Los Angeles',
 'W Los Angeles - West Beverly Hills',
 'Residence Inn by Marriott Los Angeles L.A. LIVE',
 'Courtyard by Marriott Los Angeles L.A. LIVE',
 'Holiday Inn Los Angeles - LAX Airport',
 'The Hoxton, Downtown LA',
 'Luxe Sunset Boulevard Hotel',
 'InterContinental Los Angeles Downtown',
 'Hotel Normandie',
 'Warner Center Marriott Woodland Hills',
 'Kimpton Hotel Palomar Los Angeles Beverly Hills',
 'Beverly Hills Marriott',
 'Hotel Bel-Air',
 'Century Park Hotel',
 'Los Angeles Airport Marriott',
 'Courtyard Los Angeles LAX/Century Boulevard',
 'Rotex Plaza Hotel',
 'Radisson Hotel Chatsworth',
 'Samesun Hollywood',
 'Sheraton Universa

In [182]:
def visit_hotel(df, want_to_go_name):
    each_name = want_to_go_name.split(".")
    
    for name in each_name:
        if name == each_name[0]:
            visit_html = df[["Hotel Name", "Site Link"]][df["Hotel Name"] == name]
        else:
            visit_html = visit_html.append(df[["Hotel Name", "Site Link"]][df["Hotel Name"] == name])
    
    return visit_html

In [183]:
want_to_go_name = str(input("Which hotel do you want to live?"))
visit_html = visit_hotel(df, want_to_go_name)

Which hotel do you want to live?Los Angeles Airport Marriott


In [184]:
def find_location(visit_html):
    
    location = []
    
    for link in visit_html["Site Link"]:
        html = requests.get(link)
        bsobj = soup(html.content, "lxml")
    
        for loc in bsobj.find_all("script", type = "application/ld+json"):
            if "streetAddress" in loc.string:
                find_part_html = loc.string.split("{")
                for street in find_part_html:
                    if "streetAddress" in street:
                        for value in street.split(","):
                            if "streetAddress" in value:
                                street_name = value.split(":")[1][1:][:-1] + ", "
                                   
                            if "addressLocality" in value:
                                city_name = value.split(":")[1][1:][:-1] + ", "
                                 
                            if "postalCode" in value:
                                zipcode = "CA " + value.split(":")[1][1:][:-1]
                            
        location_info = street_name + city_name + zipcode
        location.append(location_info)
    
    return location

In [185]:
location = find_location(visit_html)

In [186]:
location

['5855 West Century Boulevard, Los Angeles, CA 90045']